In [1]:
!pip install pydub librosa


[notice] A new release of pip is available: 23.1.2 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
from pydub import AudioSegment
import librosa

In [3]:
!ffmpeg -version 

ffmpeg version 2024-05-02-git-71669f2ad5-full_build-www.gyan.dev Copyright (c) 2000-2024 the FFmpeg developers
built with gcc 13.2.0 (Rev5, Built by MSYS2 project)
configuration: --enable-gpl --enable-version3 --enable-static --disable-w32threads --disable-autodetect --enable-fontconfig --enable-iconv --enable-gnutls --enable-libxml2 --enable-gmp --enable-bzlib --enable-lzma --enable-libsnappy --enable-zlib --enable-librist --enable-libsrt --enable-libssh --enable-libzmq --enable-avisynth --enable-libbluray --enable-libcaca --enable-sdl2 --enable-libaribb24 --enable-libaribcaption --enable-libdav1d --enable-libdavs2 --enable-libuavs3d --enable-libxevd --enable-libzvbi --enable-librav1e --enable-libsvtav1 --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxavs2 --enable-libxeve --enable-libxvid --enable-libaom --enable-libjxl --enable-libopenjpeg --enable-libvpx --enable-mediafoundation --enable-libass --enable-frei0r --enable-libfreetype --enable-libfribidi --enable-libharf

In [4]:
def calculate_bpm(file_path):
    """
    Calculate the BPM of an audio file using librosa.

    Args:
    file_path (str): Path to the audio file.

    Returns:
    float: Estimated BPM of the audio.
    """
    y, sr = librosa.load(file_path, sr=None)

    onset_env = librosa.onset.onset_strength(y=y, sr=sr)

    # Estimate the tempo
    tempo, _ = librosa.beat.beat_track(onset_envelope=onset_env, sr=sr)

    return tempo

In [5]:
def change_bpm(audio, original_bpm, target_bpm):
    """
    Changes the BPM of an audio file by adjusting the speed.
    This will also change the pitch of the audio.

    Args:
    audio (AudioSegment): The audio segment to adjust.
    original_bpm (float): The original BPM of the audio.
    target_bpm (float): The target BPM to achieve.

    Returns:
    AudioSegment: The modified audio segment with new BPM.
    """
    ratio = target_bpm / original_bpm

    new_audio = audio.speedup(playback_speed=ratio)

    return new_audio

In [6]:
def transition_audio(file1_path, file2_path, decibel_reduction):
    """
    Reduces the volume of the last 5 seconds of file1 and overlaps file2 over the last 5 seconds.
    
    Args:
    file1_path (str): Path to the first mp3 file.
    file2_path (str): Path to the second mp3 file.
    decibel_reduction (int): Number of decibels to reduce in the last 5 seconds of the first file.

    Returns:
    transition (AudioSegment): The combined audio segment.

    TODO: Add more variables so this can be ran as a script.
    """
    audio1 = AudioSegment.from_file(file1_path, "mp3")
    audio2 = AudioSegment.from_file(file2_path, "mp3")

    bpm_audio1 = calculate_bpm(file1_path)
    bpm_audio2 = calculate_bpm(file2_path)

    # Change the BPM of the slower audio file to match the faster audio file
    if bpm_audio1 > bpm_audio2:
        audio2 = change_bpm(audio2, bpm_audio2, bpm_audio1)
    elif bpm_audio1 < bpm_audio2:
        audio1 = change_bpm(audio1, bpm_audio1, bpm_audio2)

    # Reduce the volume of the last 5 seconds of the first audio file
    start = audio1[-17500:-10000]
    transition_seconds = audio1[-10000:].apply_gain(-decibel_reduction) # 10 seconds of transitioning
    end = audio2[:17500]

    # Combine the first part with the modified last 5 seconds
    modified_audio1 = start + transition_seconds

    # Overlay the start of the second file over the last 5 seconds of the modified first file
    transition = modified_audio1.overlay(audio2, position=-10000)

    return transition

In [7]:
audio = transition_audio("sw.mp3", "fb.mp3", 10)

In [8]:
audio.export("transition.mp3", format="mp3")

<_io.BufferedRandom name='transition.mp3'>